<img src="https://uploads6.wikiart.org/images/m-c-escher/puddle.jpg!Large.jpg" align="right"/>

## What is a Widget?
Jupyter [Widgets](https://jupyter.org/widgets) are an opinionated, limited implementation of the [Observer pattern](https://en.wikipedia.org/wiki/Observer_pattern) with a primary focus on making the state of Jupyter [Kernels](https://jupyter.org/kernels) more immediately tangible than the traditional [REPL](https://en.wikipedia.org/wiki/Read%E2%80%93eval%E2%80%93print_loop).

## Special/Not Special
Originally implemented directly in the [IPython](https://ipython.org) notebook, widgets were one of the final pieces of [The Big Split (2015)](https://blog.jupyter.org/the-big-split-9d7b88a031a7), and are one of the few notebook pieces which mostly work in Notebook Classic and JupyterLab. While once enjoying special status in the Notebook, in JupyterLab they are just like any other extension, and careful reading of the [jupyter-widgets source](https://github.com/jupyter-widgets/ipywidgets/tree/master/packages) is very instructive on how to build ecosystems on top of the JupyterLab application architecture.

## The $n$ Worlds
Widgets occupy a number of spaces:
- Kernel: widgets exist in a kernel, and typically follow the "native" observer interface
- Message: widgets implement their own protocol on top of the Jupyter message spec  
- Outputs: widgets get rendered as a media type output
- Browser: widgets are canonically represented in the browser with a Backbone-based model system
- Notebook: widgets can be serialized to JSON in the notebook

Usually all of these things are pretty separate, but what if they weren't?

In [1]:
import ipywidgets as W
import traitlets as T

In [2]:
from ipywxyz import Template, Markdown, Editor, DockBox, JSON

First, consider taking some of the traditional roles of an input, and putting them in an output.

In [3]:
flex = dict(layout=dict(flex="1"))

template_source = Editor(
"""# {{ greetings[greeting] | default("Hello") }} {{ planets["planets"][planet]["name"] | default("World") }}
{% set moons = planets["planets"][planet]["moons"] %}
{%- if moons -%}
> And also:
{% for moon in moons -%}
  - {{ moon }}
{% endfor -%}
{%- endif -%}""", 
    description="Jinja2 Template",
    **flex)
template_source

Editor(value='# {{ greetings[greeting] | default("Hello") }} {{ planets["planets"][planet]["name"] | default("…

Then think about putting the roles of a kernel library in a widget.

In [7]:
output = Editor(description="Template Output")
template = Template()
W.jsdlink((template_source, "value"), (template, "source"))
W.jsdlink((template, "value"), (output, "value"))
output

Editor(value='', description='Template Output')

At present, there is no such thing as an evented, JSON-compatible widget, so we'll kinda punt and build a custom "data" widget.

In [8]:
from urllib.request import urlopen

In [9]:
data_editor = Editor(
    urlopen("https://raw.githubusercontent.com/dariusk/corpora/master/data/science/planets.json").read().decode("utf-8"),
    description="Some planet-like things", layout=dict(max_height="60vh", height="60vh"))
json = JSON()
W.jsdlink((data_editor, "value"), (json, "source"))
data_editor

Editor(value='{\n  "description": "Planets (including dwarf planets as recognized by the IAU) that orbit the S…

In [10]:
class Context(W.Widget):
    greetings = T.List(T.Unicode(), default_value=[
        'Hi', "Howdy", "Bon Jour", "今日は"
    ]).tag(sync=True)
    greeting = T.Int(0).tag(sync=True)
    planet = T.Int(0).tag(sync=True)
    planets = T.Dict().tag(sync=True)
context = Context()
W.jsdlink((json, "value"), (context, "planets"))

In [11]:
template.context = context

There are even more useful widgets in `ipywidgets`, like nice ways to represent numbers, colors, etc.

In [12]:
greeting = W.IntSlider(description="greeting", max=len(context.greetings) - 1)
planet = W.IntSlider(description="planet", max=len(json.value["planets"]) - 1)
W.jsdlink((greeting, "value"), (context, "greeting"))
W.jsdlink((planet, "value"), (context, "planet"))
W.HBox([greeting, planet])

Finally, we can consider further transformations of existing values.

In [13]:
md = Markdown(description="a Markdown processor")
W.jsdlink((template, "value"), (md, "source"))

Which in turn can be visualized.

In [14]:
html = W.HTMLMath(discription="the final render").add_class("jp-RenderedHTMLCommon")
W.jsdlink((md, "value"), (html, "value"))
html

HTMLMath(value='', _dom_classes=('jp-RenderedHTMLCommon',))

However, things can always go wrong:

In [17]:
error = Editor("errors will appear here", description="errors be here", **flex)
W.jsdlink((template, "error"), (error, "value"))

In [18]:
error

Editor(value="attribute of type 'list' is not callable", description='errors be here', layout=Layout(flex='1')…

If you were following along at home, you've amassed quite a collection of widgets. Here they are again, all together.

In [23]:
html_box = W.VBox([greeting, planet, html])
html_box.add_traits(description=T.Unicode("Output").tag(sync=True),
                    icon_class=T.Unicode("jp-RunIcon").tag(sync=True))
hello_worlds = DockBox([data_editor, template_source, W.VBox([output, error]), html_box], 
                       layout={"height": "60vh"})
hello_worlds

DockBox(children=(Editor(value='{\n  "description": "Planets (including dwarf planets as recognized by the IAU…

If those look kinda of like JupyterLab tabs, it's because they are. Give'em a drag.